In [2]:
import ast  # for converting embeddings saved as strings back to arrays
from openai import OpenAI # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
import os # for getting API token from env variable OPENAI_API_KEY
from scipy import spatial  # for calculating vector similarities for search

# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"
OPEN_AI_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPEN_AI_KEY)
client

In [23]:
# download pre-chunked text and pre-computed embeddings
# this file is ~200 MB, so may take a minute depending on your connection speed
embeddings_path = "https://cdn.openai.com/API/examples/data/winter_olympics_2022.csv"

df = pd.read_csv(embeddings_path)
df['embedding']

0       [-0.005021067801862955, 0.00026050032465718687...
1       [0.0033927420154213905, -0.007447326090186834,...
2       [-0.00915789045393467, -0.008366798982024193, ...
3       [0.0030951891094446182, -0.006064314860850573,...
4       [-0.002936174161732197, -0.006185177247971296,...
                              ...                        
6054    [-0.027750400826334953, 0.001746018067933619, ...
6055    [-0.021714167669415474, 0.016001321375370026, ...
6056    [-0.029143543913960457, 0.014654331840574741, ...
6057    [-0.024266039952635765, 0.011665306985378265, ...
6058    [-0.021818075329065323, 0.005420385394245386, ...
Name: embedding, Length: 6059, dtype: object

In [33]:
# convert embeddings from CSV str type back to list type
df['embedding'] = df['embedding'].apply(ast.literal_eval)
df['embedding']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059 entries, 0 to 6058
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       6059 non-null   object
 1   embedding  6059 non-null   object
dtypes: object(2)
memory usage: 94.8+ KB


In [29]:
def get_query_embeddings(query):
    query_embedding_response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    return query_embedding

def strings_ranked_by_relatedness(
        query_embedding,
        df: pd.DataFrame,
        relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
        top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]


In [15]:
query_embedding = get_query_embeddings("curling gold medal")
strings, relatednesses =strings_ranked_by_relatedness(query_embedding, df)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.879


'Curling at the 2022 Winter Olympics\n\n==Medal summary==\n\n===Medal table===\n\n{{Medals table\n | caption        = \n | host           = \n | flag_template  = flagIOC\n | event          = 2022 Winter\n | team           = \n | gold_CAN = 0 | silver_CAN = 0 | bronze_CAN = 1\n | gold_ITA = 1 | silver_ITA = 0 | bronze_ITA = 0\n | gold_NOR = 0 | silver_NOR = 1 | bronze_NOR = 0\n | gold_SWE = 1 | silver_SWE = 0 | bronze_SWE = 2\n | gold_GBR = 1 | silver_GBR = 1 | bronze_GBR = 0\n | gold_JPN = 0 | silver_JPN = 1 | bronze_JPN - 0\n}}'

relatedness=0.872


"Curling at the 2022 Winter Olympics\n\n==Results summary==\n\n===Women's tournament===\n\n====Playoffs====\n\n=====Gold medal game=====\n\n''Sunday, 20 February, 9:05''\n{{#lst:Curling at the 2022 Winter Olympics – Women's tournament|GM}}\n{{Player percentages\n| team1 = {{flagIOC|JPN|2022 Winter}}\n| [[Yurika Yoshida]] | 97%\n| [[Yumi Suzuki]] | 82%\n| [[Chinami Yoshida]] | 64%\n| [[Satsuki Fujisawa]] | 69%\n| teampct1 = 78%\n| team2 = {{flagIOC|GBR|2022 Winter}}\n| [[Hailey Duff]] | 90%\n| [[Jennifer Dodds]] | 89%\n| [[Vicky Wright]] | 89%\n| [[Eve Muirhead]] | 88%\n| teampct2 = 89%\n}}"

relatedness=0.869


...

"2021 Olympic Qualification Event – Curling\n\n==Men==\n\n===Playoffs===\n\n====Qualification Game 1====\n\n''Friday, December 17, 15:00''\n{{Curlingbox\n| sheet = B\n| team1 = {{flagIOC|ITA|2022 Winter}} (Retornaz) {{X}}\n|3|0|1|0|1|0|0|1|0|0| |6\n| team2 = {{flagIOC|CZE|2022 Winter}} (Klíma)\n|0|1|0|2|0|0|1|0|1|0| |5\n}}\n{{Player percentages\n| team1 = {{flagIOC|ITA|2022 Winter}}\n| [[Simone Gonin]] | 85%\n| [[Sebastiano Arman]] | 93%\n| [[Amos Mosaner]] | 93%\n| [[Joël Retornaz]] | 90%\n| teampct1 = 90%\n| team2 = {{flagIOC|CZE|2022 Winter}}\n| [[Jiří Candra]] |88%\n| [[Radek Boháč]] | 78%\n| [[Marek Černovský]] | 68%\n| [[Lukáš Klíma (curler)|Lukáš Klíma]] | 78%\n| teampct2 = 78%\n}}"

relatedness=0.836


"2021 Olympic Qualification Event – Curling\n\n==Mixed doubles==\n\n===Round robin results===\n\n====Draw 2====\n\n''Sunday, December 5, 12:30''\n{{Curlingbox8\n| sheet = B\n| team1 = {{flagIOC|ROC|2022 Winter}} {{X}}\n|2|1|2|1|0|1|2|X| |9\n| team2 = {{flagIOC|ESP|2022 Winter}}\n|0|0|0|0|2|0|0|X| |2\n}}\n{{Curlingbox8\n| sheet = C\n| team1 = {{flagIOC|TUR|2022 Winter}} {{X}}\n|0|0|2|2|0|1|2|0| |7\n| team2 = {{flagIOC|DEN|2022 Winter}}\n|1|3|0|0|1|0|0|3| |8\n}}\n{{Curlingbox8\n| sheet = D\n| team1 = {{flagIOC|GER|2022 Winter}}\n|0|0|0|0|1|1|0|X| |2\n| team2 = {{flagIOC|HUN|2022 Winter}} {{X}}\n|1|1|2|2|0|0|2|X| |8\n}}"

relatedness=0.836


"2021 Olympic Qualification Event – Curling\n\n==Women==\n\n===Round robin results===\n\n====Draw 9====\n\n''Thursday, December 16, 14:00''\n{{Curlingbox\n| sheet = A\n| team1 = {{flagIOC|ITA|2022 Winter}} (Constantini) {{X}}\n|0|0|1|0|0|0|X|X|X|X| |1\n| team2 = {{flagIOC|GBR|2022 Winter}} (Muirhead) \n|2|0|0|1|3|2|X|X|X|X| |8\n}}\n{{Curlingbox\n| sheet = B\n| team1 = {{flagIOC|EST|2022 Winter}} (Turmann) {{X}}\n|0|1|0|0|0|0|X|X|X|X| |1\n| team2 = {{flagIOC|GER|2022 Winter}} (Jentsch)\n|0|0|1|2|1|3|X|X|X|X| |7\n}}\n{{Curlingbox\n| sheet = C\n| team1 = {{flagIOC|TUR|2022 Winter}} (Yıldız)\n|0|0|0|0|2|0|3|0|0|3| |8\n| team2 = {{flagIOC|JPN|2022 Winter}} (Fujisawa) {{X}}\n|0|0|1|0|0|2|0|1|1|0| |5\n}}\n{{Curlingbox\n| sheet = D\n| team1 = {{flagIOC|CZE|2022 Winter}} (Kubešková) {{X}}\n|2|0|1|0|1|0|2|0|0|X| |6\n| team2 = {{flagIOC|KOR|2022 Winter}} (Kim)\n|0|1|0|2|0|2|0|4|3|X| |12\n}}"

In [16]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [17]:
def query_message(
        query: str,
        df: pd.DataFrame,
        model: str,
        token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    
    query_embedding = get_query_embeddings(query)
    strings, relatednesses = strings_ranked_by_relatedness(query_embedding, df)
    introduction = 'Use the below articles on the 2022 Winter Olympics to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (
                num_tokens(message + next_article + question, model=model)
                > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


def ask(
        query: str,
        df: pd.DataFrame = df,
        model: str = GPT_MODEL,
        token_budget: int = 4096 - 500,
        print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about the 2022 Winter Olympics."},
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response.choices[0].message.content
    return response_message

In [18]:
ask('Which athletes won the gold medal in curling at the 2022 Winter Olympics?')


/Users/nardoarevalo/Desktop/pandas_learning/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


"The athletes who won the gold medal in curling at the 2022 Winter Olympics were:\n\n- Men's tournament: Team Sweden, consisting of Niklas Edin, Oskar Eriksson, Rasmus Wranå, Christoffer Sundgren, and Daniel Magnusson.\n- Women's tournament: Team Great Britain, consisting of Eve Muirhead, Vicky Wright, Jennifer Dodds, Hailey Duff, and Mili Smith.\n- Mixed doubles tournament: Team Italy, consisting of Stefania Constantini and Amos Mosaner."